In [1]:
import store
import pandas as pd
from traceback import print_exc as pe

# For testing, python will not reload a module that is already loaded
# unless you force it to
from importlib import reload
reload(store);

In [4]:
store.RebuildDB()

In [6]:
from glob import glob
import os

PATH_DATA = 'data'
PATH_TO_LOAD = os.path.join(PATH_DATA, 'sales_to_load')
PATH_LOADED = os.path.join(PATH_DATA, 'sales_loaded')

SALES_PATTERN = 'Sales_*.csv'

# Look for any new sales files to load
def GetNewData():
    '''Function will return a list of file names
       in the sales_to_load_directory'''
    file_names = glob(os.path.join(PATH_TO_LOAD, SALES_PATTERN))
    file_names.sort()
    return file_names

# For each new sales file...
def LoadNewFiles():
    file_names = GetNewData()
    for file in file_names:
        if LoadFile(file):
            # Move the file into the 'sales_loaded' directory
            # You can use os.rename for that
        else:
            print('Failed to load: ' + file)
            return False
    return True

# Load the file
def LoadFile(file_name):
    '''Function will load a single sales file into the database.
       If succesful, commit the changes, and return True.
       If any errors occur, print some detail, rollback changes and return False'''
    
    def GetCustomerID(row):
        # Check if the customer exists
        sql_check = """SELECT cust_id
                       FROM tCust
                       WHERE first = :first
                         AND last = :last
                         AND addr = :addr
                         AND zip = :zip
                         ;"""
        cust_id = pd.read_sql(sql_check, conn, params=row)
        # If no results are returned, create a record for the customer
        if len(cust_id) == 0:
            sql_insert = """
                INSERT INTO tCust (first,last,addr,zip) 
                VALUES (:first, :last, :addr, :zip);"""
            curs.execute(sql_insert, row)
        cust_id = pd.read_sql(sql_check, conn, params=row).iloc[0][0] #?
        row['cust_id'] = cust_id
        return #cust_id
    
    file = pd.read_csv(file_name, dtype={'zip':str})
    conn, curs = store.Connect()
    try:
        
        # For each row in the file...
        for row in file.to_dict(orient='records'):
            # Get or create the cust_id for the current customer
            GetCustomerID(row)
        
            # Get or create the order_id
            #order_id = GetOrderID(cust_id, date)
            GetOrderID(row)

            # Insert the data into tOrderDetail
            #
            #
    except:
        # Print detail about which file, which row, etc.
        #
        #
        conn.rollback()
        conn.close()
        return False
    
    conn.commit()
    conn.close()
    return True



def GetOrderID(row):
    # Check if order_id exists for this cust/date
    
    # If not, create a new record
    
    return order_id